## Using ak_requests

In [1]:
import common as ak
import materials, craft, stage
import numpy as np
import numpy.typing as npt
from scipy.optimize import linprog

from drop_packet import DropPacket

%load_ext autoreload
%autoreload 2

In [2]:
item_dict = ak.get_item_dict(lang="en_US", local=True)
stage_dict = ak.get_stage_dict(lang="zh_CN", local=True)
craft_dict = ak.get_craft_dict(lang="zh_CN", local=True)
psdf = ak.get_pengstats_df(server="CN")

Reading from local file
Reading from local file
Reading from local file


In [3]:
stage_san_dict, stage_names, stage_names_rev, stage_lmd_dict = stage.get_stage_info(psdf, stage_dict)
item_names, item_names_rev = ak.get_item_info(item_dict)
event_names, event_names_rev = stage.get_event_info(stage_names)

In [4]:
material_ids = ak.get_material_ids(item_names_rev, materials.MATERIAL_NAMES)

In [5]:
#main_stage_ids = ak.get_main_stage_ids(psdf)
main_stage_ids = stage.get_main_and_perm_stage_ids(psdf)
stage_ids_all = np.concatenate((main_stage_ids,[
    "wk_melee_5",
    "wk_kc_5",
    "wk_fly_5"
]))

In [6]:
craft_matrix, subprod_matrix = craft.get_craft_matrix(craft_dict, material_ids, item_names_rev)

In [7]:
drop_packet_all, coeff_matrix_all = stage.get_drop_matrix(psdf, stage_ids_all, material_ids,
                                                          stage_lmd_dict, stage_san_dict)

In [8]:
#drop_matrix, stage_san, stage_ids = stage.filter_drop_matrix(drop_matrix_all, stage_san_all, stage_ids_all)
drop_packet_all.filter_stages()

In [9]:
cutoff_threshold = 0.01
drop_packet_all.drop_matrix_cutoff(cutoff_threshold)

In [10]:
byprod_rate = 0.18
craft_constraint_matrix = ak.get_craft_constraint_matrix(craft_matrix,
                                                         subprod_matrix, byprod_rate)

In [11]:
coeff = np.sum(drop_packet_all.drop_matrix, axis=0)
#coeff = coeff_matrix_all
res = linprog(-coeff,
              A_ub=drop_packet_all.drop_matrix, b_ub=drop_packet_all.san_cost,
              A_eq=craft_constraint_matrix, b_eq=np.zeros(len(craft_constraint_matrix)),
              bounds=(0, None)
             )

print(res.success)
## Make dict from itemId -> msv at the end
msv = res.x
msv_dict = {}
for i, d in enumerate(drop_packet_all.item_ids):
    msv_dict[d] = msv[i]
    print("{}: {:.4f}".format(item_names[d], msv[i]))

True
LMD: 0.0040
Pure Gold: 9.4299
Skill Summary - 1: 1.5655
Skill Summary - 2: 3.9801
Skill Summary - 3: 10.1190
Drill Battle Record: 0.7718
Frontline Battle Record: 1.5437
Tactical Battle Record: 3.8591
Strategic Battle Record: 7.7183
Orirock: 0.8590
Orirock Cube: 2.6489
Orirock Cluster: 13.0470
Orirock Concentration: 48.7958
Damaged Device: 3.0735
Device: 9.2926
Integrated Device: 36.9731
Optimized Device: 89.7829
Ester: 1.7985
Polyester: 5.4675
Polyester Pack: 21.6723
Polyester Lump: 90.1521
Sugar Substitute: 1.8899
Sugar: 5.7418
Sugar Pack: 22.7697
Sugar Lump: 96.0446
Oriron Shard: 2.2113
Oriron: 6.7060
Oriron Cluster: 26.6266
Oriron Block: 108.5062
Diketon: 2.2699
Polyketon: 6.8817
Aketon: 27.3294
Keton Colloid: 101.3072
Loxic Kohl: 22.8702
White Horse Kohl: 79.5104
Manganese Ore: 27.2708
Manganese Trihydrate: 95.6920
Grindstone: 30.1081
Grindstone Pentahydrate: 90.3154
RMA70-12: 37.2627
RMA70-24: 87.2939
Polymerization Preparation: 244.7021
Bipolar Nanoflake: 234.8966
D32 Steel:

In [12]:
craft.print_craft_materials("Cutting Fluid Solution", craft_matrix, material_ids, item_names, item_names_rev)

LMD: 300
RMA70-12: 1
Crystalline Component: 1
Compound Cutting Fluid: 1


In [13]:
_ = stage.print_stage_drops(stage_names_rev["R8-11"], drop_packet_all, item_names)

LMD: 252
Crystalline Component: 52.79%
Orirock Cube: 36.70%
Orirock: 21.41%
Device: 13.95%
Pure Gold: 10.17%
Damaged Device: 7.96%
Orirock Cluster: 3.26%
Crystalline Circuit: 2.70%
Loxic Kohl: 2.66%
Coagulating Gel: 2.08%
Integrated Device: 1.81%


In [14]:
event_stage_ids = stage.get_event_ids("WR", psdf, event_names_rev, remove_permanent=True)
event_drop_packet, _ = stage.get_drop_matrix(psdf, event_stage_ids, material_ids,
                                         stage_lmd_dict, stage_san_dict)
print(event_drop_packet.drop_matrix)
stack_ = stage.print_stage_efficiency(event_drop_packet, msv, stage_names, item_names)
#for a, b, c in stack:
#    print("{}:\t\t{:.4}\t{}".format(stage_names[a], float(b), item_names[c]))
print(stack_)

[[108.   0.   0. ...   0.   0.   0.]
 [108.   0.   0. ...   0.   0.   0.]
 [108.   0.   0. ...   0.   0.   0.]
 ...
 [216.   0.   0. ...   0.   0.   0.]
 [216.   0.   0. ...   0.   0.   0.]
 [216.   0.   0. ...   0.   0.   0.]]
          stage_id item_id efficiency     stage_name           item_name
0       act16d5_01   30021   0.566231           WR-1    Sugar Substitute
1   act16d5_01_rep   30021   0.597321   WR-1 (rerun)    Sugar Substitute
2       act16d5_02   30031   0.520209           WR-2               Ester
3   act16d5_02_rep   30031   0.544904   WR-2 (rerun)               Ester
4       act16d5_03   30041   0.481156           WR-3        Oriron Shard
5   act16d5_03_rep   30041   0.505438   WR-3 (rerun)        Oriron Shard
6       act16d5_04   30032   0.745414           WR-4           Polyester
7   act16d5_04_rep   30032   0.752139   WR-4 (rerun)           Polyester
8       act16d5_05   30052   0.847881           WR-5           Polyketon
9   act16d5_05_rep   30052   0.775045   WR

In [16]:
stages_ = stage.get_stages_which_drop("Compound Cutting Fluid", drop_packet_all.drop_matrix,
                                      material_ids, stage_ids_all, item_names_rev)
for s, d in stages_:
        print("{}: {:.4f}%".format(stage_names[s], 100*d))


10-17: 57.4899%
SN-1 (permanent): 27.2727%
9-6: 27.1794%
10-15: 1.8682%
10-14: 1.2518%
7-14: 1.2308%
4-8: 1.1294%
4-5: 1.1245%
9-2: 1.1129%
9-11: 1.0638%
6-4: 1.0582%
9-12: 1.0204%
9-19: 1.0126%


In [24]:
def get_required_sanity(drop_packet: DropPacket, item_id: str, stage_id: str, n_items: int) -> float:
    espd = drop_packet.get_espd(item_id, stage_id)
    total_sanity = espd * n_items
    return total_sanity
    

In [29]:
san = get_required_sanity(event_drop_packet, item_names_rev["Incandescent Alloy"],
                          stage_names_rev["WR-8 (rerun)"], 141)
daily_san = 240 + 60
days_left = 7
remaining_sanity = san - days_left*daily_san
print("Remaining sanity: {}".format(remaining_sanity))

Remaining sanity: 1509.1409166321027
